In [ ]:
import numpy as np
import re

In [ ]:
def loadPCAWeightsFromSV(fName,nbits=12):
    pca_weights=np.zeros(30*1024).reshape(30,32,32)

    regex_in=re.compile(f"^assign weights_in\[(\d*)\]\[(\d*)\]\[(\d*)\] = {nbits}'b(\d*);")

    with open(fName,'r') as _file:
        for line in _file:
            m=regex_in.match(line)
            if m:
                i,j,k,v=m.groups()
                v=int(v,2)
                v = (-1*(v>>11)*2**11 + (v&0x7ff))/2**11
                pca_weights[int(i)][int(j)][int(k)]=v

    return pca_weights

In [ ]:
def loadPCAInputsFromSV(fName):
    pcaInputs=np.zeros(1024).reshape(32,32)

    regex_in=re.compile("^assign pca_data_in_10b\[(\d*)\]\[(\d*)\] = 10'b(\d*);")

    with open(fName,'r') as _file:
        for line in _file:
            m=regex_in.match(line)
            if m:
                i,j,v=m.groups()
                v=int(v,2)
                pcaInputs[int(i)][int(j)]=v

    return pcaInputs

In [ ]:
def toTwosCompliment(x,N=11,M=0,doRounding=False):
    if doRounding:
        y=round(x*2**N)
    else:
        y=int(x*2**N)
    if y < 0:
        return f"{M+N+1}'b1"+f"{(2**(N+M)+y):0b}".zfill(M+N)
    else:
        return f"{M+N+1}'b{y:0b}"

toTwosCompliment=np.vectorize(toTwosCompliment)

In [ ]:
R_inv_float=np.load('mandrill_140/inv_R_matrix_float32.npy')[:,:30]*32


In [ ]:
pcaWeights=loadPCAWeightsFromSV('svData/PCA_weightsInput_Original.sv')
pcaWeights=pcaWeights.reshape(30,1024).T


In [ ]:
pcaInputs=loadPCAInputsFromSV('svData/testVectors_0.sv')


In [ ]:
np.matmul(pcaInputs.flatten(),pcaWeights)

In [ ]:
np.matmul(pcaInputs.flatten(),pcaWeights)*2**11

In [ ]:
output=((np.matmul(pcaInputs.flatten(),pcaWeights)*2**11).astype(int)>>10)
output

In [ ]:
twosOutput=toTwosCompliment(output,0,6)
twosOutput

In [ ]:
for n in range(0,601,100):
    lines = ""
    with open(f'svData/testVectors_{n}.sv','r') as _file:
        #drop old 19 b pca values
        for l in _file:
            if '// 19b PCA outputs' in l:
                continue
            if 'output from original PCA weights' in l:
                continue
            if 'pca_output' in l:
                continue
            lines += l

    print(n)
    pcaInputs=loadPCAInputsFromSV(f'svData/testVectors_{n}.sv')
    
    output7=((np.matmul(pcaInputs.flatten(),pcaWeights)*2**11).astype(int)>>10)
    twosOutput=toTwosCompliment(output7,0,6)
    
    lines+="// 7-bit output from original PCA weights (from Panpan)\n\n"
    for i,v in enumerate(twosOutput):
        lines += f"assign pca_output_orig_7b[{i}] = {v};\n"
    lines += '\n\n'
    
    output19=((np.matmul(pcaInputs.flatten(),pcaWeights)*2**11).astype(int))
    twosOutput=toTwosCompliment(output19,0,18)
    
    lines+="// 19-bit output from original PCA weights (from Panpan)\n\n"
    for i,v in enumerate(twosOutput):
        lines += f"assign pca_output_orig_19b[{i}] = {v};\n"
    lines += '\n\n'
    
    with open(f'svData/testVectors_{n}.sv','w') as _file:
        _file.write(lines)

    l=', '.join([f'{x/2:.1f}' for x in output7])
    with open(f'svData/dataOut_{n}_Q7.6.dat','w') as _file:
        _file.write(l)

    l=', '.join([f'{x/2**11:.11f}' for x in output19])
    with open(f'svData/dataOut_{n}_Q19.8.dat','w') as _file:
        _file.write(l)

In [ ]:
#generate weights data file as floats
lines = ""
for l in pcaWeights.T:
    lines += ', '.join([str(x) for x in l])  + "\n"
with open('svData/PCA_weightsInput_Original.dat','w') as _file:
    _file.write(lines[:-1])